# **Deep Learning and CNN for Computer Vision, Hokkaido University**

## Day 3, Notebook -1: Object Detection using YoloV5

In this part of the session you will be exploring how to use the YOLO v5 object detector


So lets get started!


We will use the https://github.com/ultralytics/yolov5 repository for this tutorial

Reference: https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb

# Step 1: Clone github repository

#### **Connect to Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

#### **Custom Dataset Setup (We will use the Kangaroo dataset)**

In [ ]:
# Change path to you google drive location of the dataset as needed
!unzip '/content/gdrive/MyDrive/Hokkaido Uni DL/Day-3/yoloV5_kangaroo.zip'

#### **Clone the Github Repo (Default location /content/)**

In [ ]:
# By Default Cloned under /content/
!git clone https://github.com/ultralytics/yolov5

Install the required libraries

In [ ]:
%cd yolov5
from yolov5 import utils
#display = utils.notebook_init()  # checks Pytorch and GPU used. Need changes to correct path
!pip install -r requirements.txt

#### **Quick Reference on YAML**

#### [**What is YAML?**](https://circleci.com/blog/what-is-yaml-a-beginner-s-guide/)

YAML --> **Y**et **A**nother **M**arkup **L**anguage

YAML is a digestible data serialization language often used to create configuration files with any programming language.

Designed for human interaction, YAML is a strict superset of JSON, another data serialization language. But because it’s a strict superset, it can do everything that JSON can and more. One major difference is that newlines and indentation actually mean something in YAML, as opposed to JSON, which uses brackets and braces.

Reference and Source: https://circleci.com/blog/what-is-yaml-a-beginner-s-guide/

# Step 2: Setup Training Configuration

In [ ]:
#@title Setup Training YAML File
number_of_classes = 1 #@param {type:"integer"}
with open('new_train_yaml', 'w+') as file:
    file.write(
        f"""
        # parameters
        nc: {number_of_classes}  # number of classes
        depth_multiple: 0.33  # model depth multiple
        width_multiple: 0.50  # layer channel multiple

        # anchors
        anchors:
          - [10,13, 16,30, 33,23]  # P3/8
          - [30,61, 62,45, 59,119]  # P4/16
          - [116,90, 156,198, 373,326]  # P5/32

        # YOLOv5 backbone
        backbone:
          # [from, number, module, args]
          [[-1, 1, Focus, [64, 3]],  # 0-P1/2
           [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
           [-1, 3, BottleneckCSP, [128]],
           [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
           [-1, 9, BottleneckCSP, [256]],
           [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
           [-1, 9, BottleneckCSP, [512]],
           [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
           [-1, 1, SPP, [1024, [5, 9, 13]]],
           [-1, 3, BottleneckCSP, [1024, False]],  # 9
          ]

        # YOLOv5 head
        head:
          [[-1, 1, Conv, [512, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 6], 1, Concat, [1]],  # cat backbone P4
           [-1, 3, BottleneckCSP, [512, False]],  # 13

           [-1, 1, Conv, [256, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 4], 1, Concat, [1]],  # cat backbone P3
           [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

           [-1, 1, Conv, [256, 3, 2]],
           [[-1, 14], 1, Concat, [1]],  # cat head P4
           [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

           [-1, 1, Conv, [512, 3, 2]],
           [[-1, 10], 1, Concat, [1]],  # cat head P5
           [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

           [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
          ]
        """
    )

# Step 3: Setup Dataset paths

In [ ]:
#@title Setup Dataset Configuration (Data.yaml)
train_data_dir = "/content/yolo_kangaroo/Kangaroo/train" #@param {type:"string"}
val_data_dir = "/content/yolo_kangaroo/Kangaroo/valid" #@param {type:"string"}
class_names = ['kangaroo'] #@param {type:"raw"}
with open('new_data_yaml', 'w+') as file:
    file.write(
        f"""
        train: {train_data_dir}
        val: {val_data_dir}

        nc: {number_of_classes}
        names: {class_names}
        """
    )

# Step 4: Start Training

**Configuation to try:**

*   **Image Size**: 416
*   **Batch Size**: 16
*   **Epochs**: 200
*   **Data Source details** : new_data_yaml (Created earlier)
*   **Training details** : new_train_yaml (Created earlier)

Example:

!python /content/yolov5/train.py --**img** 416 --**batch** 16 --**epochs** 300 --**data** /content/yolov5/new_data_yaml --**cfg** /content/yolov5/new_train_yaml


In [ ]:
!python /content/yolov5/train.py --img 416 --batch 16 --epochs 200 --data /content/yolov5/new_data_yaml --cfg /content/yolov5/new_train_yaml

# Step 5: Test the trained model on sample images

In [ ]:
!ls /content/yolov5/runs/train/exp

### **Inference/Testing**
'detect.py' is used to run YOLOv5 testing/inference on different types of inputs such as: image, video, webcam input, directory glob, Youtube, RTSP/RTMP/HTTP Stream.

**Usage Syntax:**

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image
                          vid.mp4  # video
                          path/  # directory
                          path/*.jpg  # glob
                          'https://youtu.beZgi9g1ksQHc'  # YouTube
                          'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
                 --weights path/to/weights
                 --img 416
                 --conf 0.5
                 --save-txt
```

In [ ]:
!python /content/yolov5/detect.py --source '/content/yolo_kangaroo/Kangaroo/valid/images/00010.jpg' --weights '/content/yolov5/runs/train/exp/weights/best.pt' --img 416 --conf 0.5 --save-txt
!python /content/yolov5/detect.py --source '/content/yolo_kangaroo/Kangaroo/valid/images/00100.jpg' --weights '/content/yolov5/runs/train/exp/weights/best.pt' --img 416 --conf 0.5 --save-txt
!python /content/yolov5/detect.py --source '/content/yolo_kangaroo/Kangaroo/valid/images/00109.jpg' --weights '/content/yolov5/runs/train/exp/weights/best.pt' --img 416 --conf 0.5 --save-txt

# Step 6: Display result images

In [ ]:
import cv2
from matplotlib import pyplot as plt
from PIL import Image

# This is needed to display the images.
%matplotlib inline

In [ ]:
image = Image.open('/content/yolov5/runs/detect/exp/00010.jpg')
plt.imshow(image)

In [ ]:
image = Image.open('/content/yolov5/runs/detect/exp2/00100.jpg')
plt.imshow(image)

In [ ]:
image = Image.open('/content/yolov5/runs/detect/exp3/00109.jpg')
plt.imshow(image)

### Try on a Youtube video (Optional, takes more time!)

In [ ]:
# Interrupt runtime after 1 min if the video is too long
#!python detect.py --source 'https://www.youtube.com/watch?v=wqctLW0Hb_0'
!python detect.py --source 'https://youtu.be/KBsqQez-O4w'

In [ ]:
# Convert mp4 to webm
!ffmpeg -i /content/yolov5/runs/detect/exp7/KBsqQez-O4w.mp4 -vcodec vp9 ./DetectionResult.webm

In [ ]:
# Check the Result!!
import io
from base64 import b64encode
from IPython.display import HTML

with  io.open('/content/yolov5/DetectionResult.webm','r+b') as f:
    mp4 = f.read()
data_url = "data:video/webm;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/webm">
</video>
""" % data_url)

# Step 7: Display performance analysis

Training results are automatically logged to Tensorboard and CSV as `results.csv`, which is plotted as `results.png` (below) after training completes. You can also plot any `results.csv` file manually:

In [ ]:
image = Image.open('/content/yolov5/runs/train/exp/results.png') # Change 'exp' to the last in the train directory
plt.imshow(image)

In [ ]:
#[OPTIONAL] Alternate way of ploting the curves from CSV file
from utils.plots import plot_results
# Change the path to the last exp under train folder.
plot_results('/content/yolov5/runs/train/exp/results.csv')  # plot 'results.csv' as 'results.png'